#### **Solicitud traspaso sucursales**

##### Subida de archivos

In [1]:
import pandas as pd

# Cargar los archivos CSV en dataframes con una codificación diferente
df1 = pd.read_csv(r"C:\Users\constanza.perez\Downloads\SugeridoTransferencias(Data ).csv", encoding='latin1')
df2 = pd.read_csv(r"C:\Users\constanza.perez\Downloads\Stock 08-ene 2025(Sheet1).csv", encoding='latin1')

# Mostrar los primeros 5 registros de cada dataframe
print("Primer dataframe:")
print(df1.head())

print("\nSegundo dataframe:")
print(df2.head())


Primer dataframe:
  Punto de venta - origen Código ERP - origen Punto de venta - destino  \
0   COLLOKY 15 NORTE VIÑA                C560          COLLOKY AHUMADA   
1   COLLOKY 15 NORTE VIÑA                C560          COLLOKY AHUMADA   
2   COLLOKY 15 NORTE VIÑA                C560          COLLOKY AHUMADA   
3   COLLOKY 15 NORTE VIÑA                C560          COLLOKY AHUMADA   
4   COLLOKY 15 NORTE VIÑA                C560          COLLOKY AHUMADA   

  Código ERP - destino Código producto  \
0                 C110   CESX0150B2V25   
1                 C110   JGPY0230TUV25   
2                 C110   P1JE12092AV25   
3                 C110   P1JE3182SMV25   
4                 C110   P1JE42976AV25   

                                Descripción Tipo traspaso  \
0   CESX01 CALZONCILLO K1 BOY, AZUL, 12ANOS             N   
1   JGPY02 JUGUETE NB, ROSADOS, TALLA UNICA             N   
2  P1JE12 POLERA IN GI, OFF WHITE, 24 MESES             N   
3  P1JE31 POLERA IN BO, ROJO, S (3-6 MES

C:\Users\constanza.perez\AppData\Local\Temp\ipykernel_14272\3940357283.py:5: DtypeWarning: Columns (19,21,23,24,26) have mixed types. Specify dtype option on import or set low_memory=False.
  df2 = pd.read_csv(r"C:\Users\constanza.perez\Downloads\Stock 08-ene 2025(Sheet1).csv", encoding='latin1')


##### Procemos a eliminar espacios en blanco que entorpecen el merge. Posteriormente, realizamos cruce para determinar si hay igualdad entre stock df1 y stock df2.

In [6]:
import pandas as pd

# Supongamos que ya has cargado los dataframes df1 y df2

# Función para eliminar los espacios en todas las columnas de un dataframe
def eliminar_espacios(df):
    df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)
    return df

# Eliminamos los espacios en df1 y df2
df1 = eliminar_espacios(df1)
df2 = eliminar_espacios(df2)

# Aseguramos que las columnas que vamos a usar para hacer la comparación son del mismo tipo (por ejemplo, cadenas de texto)
df1['Código ERP - origen'] = df1['Código ERP - origen'].astype(str)
df1['Código producto'] = df1['Código producto'].astype(str)
df2['ID Sucursal'] = df2['ID Sucursal'].astype(str)
df2['ID Producto'] = df2['ID Producto'].astype(str)

# Convertimos las columnas de stock a tipo numérico
df1['Stock actual Punto de venta origen'] = pd.to_numeric(df1['Stock actual Punto de venta origen'], errors='coerce')
df2['Stock Libre Utilizacion'] = pd.to_numeric(df2['Stock Libre Utilizacion'], errors='coerce')

# Hacemos un merge de los dos dataframes basándonos en las columnas correspondientes
df_merged = pd.merge(df1, df2, left_on=['Código ERP - origen', 'Código producto'], right_on=['ID Sucursal', 'ID Producto'])

# Comparamos los valores de 'Stock actual Punto de venta origen' y 'Stock Libre Utilizacion'
df_merged['Son iguales'] = df_merged['Stock actual Punto de venta origen'] == df_merged['Stock Libre Utilizacion']

# Seleccionamos las columnas relevantes para los resultados
df_resultados = df_merged[['Código ERP - origen', 'Código producto', 'Stock actual Punto de venta origen', 'Stock Libre Utilizacion', 'Son iguales']]

# Mostramos los resultados
print(df_resultados)


C:\Users\constanza.perez\AppData\Local\Temp\ipykernel_14272\3696569570.py:7: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)


      Código ERP - origen Código producto  Stock actual Punto de venta origen  \
0                    C560   CESX0150B2V25                                   6   
1                    C560   JGPY0230TUV25                                   3   
2                    C560   P1JE12092AV25                                   1   
3                    C560   P1JE3182SMV25                                   2   
4                    C560   P1JE42976AV25                                   2   
...                   ...             ...                                 ...   
17024                O552   PTPO750609S25                                   3   
17025                O552   STPU308406S25                                   2   
17026                O552   STSX26303AS25                                   8   
17027                O552   TBLY575706S25                                   4   
17028                O552   VEBY28103AS25                                   4   

       Stock Libre Utilizac

In [7]:
# Contamos las filas que tienen 'Son iguales' igual a True y False
conteo_true = df_resultados['Son iguales'].sum()
conteo_false = len(df_resultados) - conteo_true
total_filas = len(df_resultados)

# Calculamos los porcentajes
porcentaje_true = (conteo_true / total_filas) * 100
porcentaje_false = (conteo_false / total_filas) * 100

# Creamos un DataFrame con los resultados
df_conteo = pd.DataFrame({
    'Resultado': ['True', 'False'],
    'Cantidad': [conteo_true, conteo_false],
    'Porcentaje': [porcentaje_true, porcentaje_false]
})

# Mostramos los resultados
print(df_conteo)


  Resultado  Cantidad  Porcentaje
0      True     17013   99.906043
1     False        16    0.093957


##### Hallazgo de 16 filas con stock diferente que posteriormente serán excluidas.

In [8]:
# Filtrar el DataFrame para obtener solo las filas donde 'Son iguales' es False
df_false = df_resultados[df_resultados['Son iguales'] == False]

# Mostramos los resultados
print(df_false)


      Código ERP - origen Código producto  Stock actual Punto de venta origen  \
9681                 C105   P1JE31823AV25                                   2   
9814                 C105   P1JE31823AV25                                   2   
10186                C318   P1JE4297B2V25                                  12   
10196                C318   P1JE4297B2V25                                  12   
10316                C318   P1JE4297B2V25                                  12   
10432                C318   P1JE4297B2V25                                  12   
10498                C318   P1JE4297B2V25                                  12   
10528                C318   P1JE4297B2V25                                  12   
10578                C318   P1JE4297B2V25                                  12   
10660                C318   P1JE4297B2V25                                  12   
14564                O157   FATU340912S25                                  18   
14579                O157   

##### Agrupación de sucursales con salidas y conteo único de SKU

In [17]:
import pandas as pd

# Paso 1: Agrupar por 'Código ERP - origen' y luego por 'Código ERP - destino'
grupos_origen = df1.groupby(['Código ERP - origen', 'Código ERP - destino'])

# Paso 2: Contar las filas donde el 'Código ERP - destino' repite el 'Código ERP - origen'
# Convertimos los grupos a un DataFrame temporal para facilitar la comprobación
df_temp = grupos_origen.size().reset_index(name='counts')
repeticiones = df_temp[df_temp['Código ERP - destino'] == df_temp['Código ERP - origen']]
print(f"Número de filas donde 'Código ERP - destino' repite el 'Código ERP - origen': {len(repeticiones)}")

# Paso 3: Obtener 'Código ERP - destino' único con la suma de 'Salidas Totales' y el conteo único de 'Código producto'
resultado = grupos_origen.agg({
    'Salidas Totales': 'sum',
    'Código producto': pd.Series.nunique
}).reset_index()

# Paso 4: Renombrar las columnas para claridad
resultado = resultado.rename(columns={'Código producto': 'Conteo único de Código producto'})

# Paso 5: Guardar el resultado en un archivo Excel en la ruta especificada
ruta_excel = r'C:\Users\constanza.perez\OneDrive - Colgram\Escritorio\resultados_agrupados.xlsx'
resultado.to_excel(ruta_excel, index=False)


Número de filas donde 'Código ERP - destino' repite el 'Código ERP - origen': 0
El archivo Excel ha sido guardado en: C:\Users\constanza.perez\OneDrive - Colgram\Escritorio\resultados_agrupados.xlsx


##### Aplicación clustering para obtención de número optimo suscursales. Minimizar SKU y maximizar salidas.

In [38]:
import pandas as pd
from sklearn.cluster import KMeans
import numpy as np
from sklearn.preprocessing import LabelEncoder

# Cargar el archivo Excel generado previamente
ruta_excel = r'C:\Users\constanza.perez\OneDrive - Colgram\Escritorio\resultados_agrupados.xlsx'
df = pd.read_excel(ruta_excel)

# Convertir 'Código ERP - destino' a valores numéricos
label_encoder = LabelEncoder()
df['Código ERP - destino'] = label_encoder.fit_transform(df['Código ERP - destino'])

# Asegurar que las columnas necesarias estén presentes
if 'Conteo único de Código producto' not in df.columns or 'Salidas Totales' not in df.columns:
    raise ValueError("El DataFrame no contiene las columnas necesarias.")

# Función para determinar el punto de inflexión en el método del codo
def encontrar_punto_de_inflexion(wcss):
    if len(wcss) < 3:
        return 1  # Si hay menos de 3 puntos, devolvemos 1 cluster
    dif1 = np.diff(wcss)
    dif2 = np.diff(dif1)
    return np.argmin(dif2) + 1

# Para cada 'Código ERP - origen', aplicar K-means clustering y determinar el número óptimo de clusters
def aplicar_kmeans(grupo):
    X = grupo[['Código ERP - destino', 'Conteo único de Código producto', 'Salidas Totales']].values
    max_clusters = min(10, len(X))
    if max_clusters < 2:
        return pd.Series({'Número de clusters óptimo': 1})
    
    wcss = []
    
    # Determinar el número óptimo de clusters usando el método del codo
    for i in range(1, max_clusters + 1):
        kmeans = KMeans(n_clusters=i, init='k-means++', max_iter=300, n_init=10, random_state=0)
        kmeans.fit(X)
        wcss.append(kmeans.inertia_)
    
    # Encontrar el punto de inflexión
    n_clusters_optimo = encontrar_punto_de_inflexion(wcss)
    n_clusters_optimo = min(n_clusters_optimo, len(X))  # Asegurar que el número de clusters no sea mayor que el número de muestras
    
    return pd.Series({'Número de clusters óptimo': n_clusters_optimo})

# Aplicar la función a cada grupo de 'Código ERP - origen'
resultado_clusters = df.groupby('Código ERP - origen').apply(aplicar_kmeans).reset_index()

# Guardar los resultados en un nuevo archivo Excel
ruta_excel_clusters = r'C:\Users\constanza.perez\OneDrive - Colgram\Escritorio\resultado_clusters_optimos.xlsx'
resultado_clusters.to_excel(ruta_excel_clusters, index=False)

print(f"El archivo Excel con el número de clusters óptimos ha sido guardado en: {ruta_excel_clusters}")


C:\Users\constanza.perez\AppData\Local\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1429: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(
C:\Users\constanza.perez\AppData\Local\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1429: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(
C:\Users\constanza.perez\AppData\Local\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1429: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(
C:\Users\constanza.perez\AppData\Local\anaconda3\Lib\site-packag

El archivo Excel con el número de clusters óptimos ha sido guardado en: C:\Users\constanza.perez\OneDrive - Colgram\Escritorio\resultado_clusters_optimos.xlsx


C:\Users\constanza.perez\AppData\Local\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1429: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(
C:\Users\constanza.perez\AppData\Local\Temp\ipykernel_14272\3967975700.py:48: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  resultado_clusters = df.groupby('Código ERP - origen').apply(aplicar_kmeans).reset_index()


##### Obtenemos que sucursales de destino serán asociadas a cada sucursal de origen en base a número óptimo previamente obtenido. Prioridad en aquellas que minimizan SKU.

In [39]:
import pandas as pd

# Cargar los archivos Excel generados previamente
ruta_excel_clusters = r'C:\Users\constanza.perez\OneDrive - Colgram\Escritorio\resultado_clusters_optimos.xlsx'
ruta_excel_agrupados = r'C:\Users\constanza.perez\OneDrive - Colgram\Escritorio\resultados_agrupados.xlsx'

df_clusters = pd.read_excel(ruta_excel_clusters)
df_agrupados = pd.read_excel(ruta_excel_agrupados)

# Supongamos que ya has cargado df_false
# Filtrar los códigos ERP origen que no están en df_false
codigos_erp_origen_excluir = df_false['Código ERP - origen'].unique()
df_clusters_filtrado = df_clusters[~df_clusters['Código ERP - origen'].isin(codigos_erp_origen_excluir)]

# Para cada código ERP origen único, seleccionar la cantidad de Código ERP - destino basado en el número óptimo de clusters
resultados_finales = []

for _, row in df_clusters_filtrado.iterrows():
    codigo_erp_origen = row['Código ERP - origen']
    n_clusters_optimo = row['Número de clusters óptimo']
    
    df_origen = df_agrupados[df_agrupados['Código ERP - origen'] == codigo_erp_origen]
    
    # Ordenar filas para minimizar el conteo único de código de producto
    df_origen_ideal = df_origen.sort_values('Conteo único de Código producto').head(n_clusters_optimo)
    
    # Si no hay suficientes filas que cumplan la condición, aplicar estrategia alternativa
    if len(df_origen_ideal) < n_clusters_optimo:
        df_origen_ideal = pd.concat([df_origen_ideal, df_origen[~df_origen.index.isin(df_origen_ideal.index)]])
        df_origen_ideal = df_origen_ideal.sort_values('Conteo único de Código producto').head(n_clusters_optimo)
    
    resultados_finales.append(df_origen_ideal)

# Concatenar todos los resultados en un solo DataFrame
df_resultados_finales = pd.concat(resultados_finales, ignore_index=True)

# Guardar los resultados en un nuevo archivo Excel
ruta_excel_resultados_finales = r'C:\Users\constanza.perez\OneDrive - Colgram\Escritorio\resultados_finales.xlsx'
df_resultados_finales.to_excel(ruta_excel_resultados_finales, index=False)

print(f"El archivo Excel con los resultados finales ha sido guardado en: {ruta_excel_resultados_finales}")


El archivo Excel con los resultados finales ha sido guardado en: C:\Users\constanza.perez\OneDrive - Colgram\Escritorio\resultados_finales.xlsx
